In [8]:
import easyocr
import cv2
import os

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

# Directory containing images
image_dir = 'data'

def extract_text(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    # Perform OCR using EasyOCR
    result = reader.readtext(image)
    # Combine the detected text into a single string
    text = ' '.join([item[1] for item in result])
    return text

# Iterate over all images in the directory
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        img_path = os.path.join(image_dir, filename)
        extracted_text = extract_text(img_path)
        print(f"Extracted text from {filename}:")
        print(extracted_text)
        print("-" * 50)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteExtracted text from Screenshot 2025-03-04 at 12.29.48.png:
Edai 20M1lZ Aclara: SCMi412-8 oudlel TOU Rate recoh 17013 kwh Dnv0s Iq1OJiA S0Hz 2020   JKB 001024020118CAOC
--------------------------------------------------
Extracted text from Screenshot 2025-03-04 at 12.30.45.png:
^4 Landls_ E470 Herdtealiet Lid AtcueFL5 & B U TotaL Acl Imtorl 45 76Lw4 Tp/kWh CIB 7C153 0+521 I000 (n-Sn L8C0L6 100a Z30v 50hz Caida-vi P5? Property Kacquarie Entro 255 BGSMLL02430780 0120/5G50226 Cemi8 0120 FLAT 501 Soi 56"0
--------------------------------------------------


Try 2

In [10]:
import os
import re
import cv2
import numpy as np
import easyocr

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

# Directory containing images
image_dir = 'data'

def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    # Apply adaptive thresholding with a larger block size
    processed = cv2.adaptiveThreshold(enhanced, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      cv2.THRESH_BINARY_INV, 21, 10)
    
    return processed

def extract_meter_reading(image_path):
    processed_image = preprocess_image(image_path)
    # Perform OCR with EasyOCR, allowing only numbers and 'kWh'
    result = reader.readtext(processed_image, allowlist='0123456789.kWh')
    # Combine detected text into a single string
    full_text = ' '.join([text for _, text, _ in result])
    # Use regex to find the number preceding 'kWh'
    match = re.search(r'(\d+\.?\d*)\s*kWh', full_text)
    if match:
        return match.group(1)
    else:
        return "Meter reading not found"

# Iterate over all images in the directory
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        img_path = os.path.join(image_dir, filename)
        meter_reading = extract_meter_reading(img_path)
        print(f"Meter reading from {filename}: {meter_reading}")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Meter reading from Screenshot 2025-03-04 at 12.29.48.png: Meter reading not found
Meter reading from Screenshot 2025-03-04 at 12.30.45.png: Meter reading not found
